<img src="files/model_1_outline.png">

# change log
### Epoch = 800
### Shuffle=False


sequence generator shape (corrected) (b, n, w, h, c)

In [1]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [5]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [6]:
# some global params
SIZE = (128, 128) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [7]:
# pattern to get videos and classes
glob_pattern='../cholec80/model_data/{classname}/*.avi'


In [8]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [9]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class CalotTriangleDissection, validation count: 170, train count: 347
class CleaningCoagulation, validation count: 170, train count: 347
class ClippingCutting, validation count: 170, train count: 347
class GallbladderDissection, validation count: 170, train count: 347
class GallbladderPackaging, validation count: 170, train count: 347
class GallbladderRetraction, validation count: 170, train count: 347
class Preparation, validation count: 170, train count: 347
Total data: 7 classes for 2429 files for train


In [10]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 1190 files for validation


In [11]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [12]:
#valid.files

## BUILD CONV NET

In [13]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from keras.applications.mobilenet import MobileNet
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
        
    # adding a max pool
    output = GlobalMaxPool2D()

    return keras.Sequential([model, output])

## Build GRU

In [14]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [15]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 128, 128, 3)


## epochs, call backs

In [16]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210616-105556


In [18]:
EPOCHS=800
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/800
303/303 [==============================] - 170s 560ms/step - loss: 1.9809 - acc: 0.1419 - val_loss: 1.9461 - val_acc: 0.1427

Epoch 00001: saving model to model2_3_1_chkp\weights.01-1.95.hdf5
Epoch 2/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9598 - acc: 0.1221 - val_loss: 1.9477 - val_acc: 0.1191

Epoch 00002: saving model to model2_3_1_chkp\weights.02-1.95.hdf5
Epoch 3/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9546 - acc: 0.1287 - val_loss: 1.9470 - val_acc: 0.1436

Epoch 00003: saving model to model2_3_1_chkp\weights.03-1.95.hdf5
Epoch 4/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9499 - acc: 0.1110 - val_loss: 1.9464 - val_acc: 0.1436

Epoch 00004: saving model to model2_3_1_chkp\weights.04-1.95.hdf5
Epoch 5/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9499 - acc: 0.1357 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00005: saving model to model2_3_1_chkp\weights.05-1.

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00040: saving model to model2_3_1_chkp\weights.40-1.95.hdf5
Epoch 41/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00041: saving model to model2_3_1_chkp\weights.41-1.95.hdf5
Epoch 42/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00042: saving model to model2_3_1_chkp\weights.42-1.95.hdf5
Epoch 43/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00043: saving model to model2_3_1_chkp\weights.43-1.95.hdf5
Epoch 44/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00044: saving model to model2_3_1_chkp\weights.44-1.95.hdf5
Epo

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00079: saving model to model2_3_1_chkp\weights.79-1.95.hdf5
Epoch 80/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00080: saving model to model2_3_1_chkp\weights.80-1.95.hdf5
Epoch 81/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00081: saving model to model2_3_1_chkp\weights.81-1.95.hdf5
Epoch 82/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00082: saving model to model2_3_1_chkp\weights.82-1.95.hdf5
Epoch 83/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1444 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00083: saving model to model2_3_1_chkp\weights.83-1.95.hdf5
Epo

Epoch 118/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00118: saving model to model2_3_1_chkp\weights.118-1.95.hdf5
Epoch 119/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1427 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00119: saving model to model2_3_1_chkp\weights.119-1.95.hdf5
Epoch 120/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00120: saving model to model2_3_1_chkp\weights.120-1.95.hdf5
Epoch 121/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00121: saving model to model2_3_1_chkp\weights.121-1.95.hdf5
Epoch 122/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00122: saving model to model2_3_1_chkp\we


Epoch 00156: saving model to model2_3_1_chkp\weights.156-1.95.hdf5
Epoch 157/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00157: saving model to model2_3_1_chkp\weights.157-1.95.hdf5
Epoch 158/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00158: saving model to model2_3_1_chkp\weights.158-1.95.hdf5
Epoch 159/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00159: saving model to model2_3_1_chkp\weights.159-1.95.hdf5
Epoch 160/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00160: saving model to model2_3_1_chkp\weights.160-1.95.hdf5
Epoch 161/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.945

...

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00626: saving model to model2_3_1_chkp\weights.626-1.95.hdf5
Epoch 627/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00627: saving model to model2_3_1_chkp\weights.627-1.95.hdf5
Epoch 628/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00628: saving model to model2_3_1_chkp\weights.628-1.95.hdf5
Epoch 629/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00629: saving model to model2_3_1_chkp\weights.629-1.95.hdf5
Epoch 630/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00630: saving model to model2_3_1_chkp\weights.630-1.95

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00666: saving model to model2_3_1_chkp\weights.666-1.95.hdf5
Epoch 667/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1440 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00667: saving model to model2_3_1_chkp\weights.667-1.95.hdf5
Epoch 668/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1427 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00668: saving model to model2_3_1_chkp\weights.668-1.95.hdf5
Epoch 669/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00669: saving model to model2_3_1_chkp\weights.669-1.95.hdf5
Epoch 670/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00670: saving model to model2_3_1_chkp\weights.670-1.95

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00706: saving model to model2_3_1_chkp\weights.706-1.95.hdf5
Epoch 707/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1440 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00707: saving model to model2_3_1_chkp\weights.707-1.95.hdf5
Epoch 708/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00708: saving model to model2_3_1_chkp\weights.708-1.95.hdf5
Epoch 709/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00709: saving model to model2_3_1_chkp\weights.709-1.95.hdf5
Epoch 710/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1427 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00710: saving model to model2_3_1_chkp\weights.710-1.95

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00746: saving model to model2_3_1_chkp\weights.746-1.95.hdf5
Epoch 747/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1440 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00747: saving model to model2_3_1_chkp\weights.747-1.95.hdf5
Epoch 748/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00748: saving model to model2_3_1_chkp\weights.748-1.95.hdf5
Epoch 749/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00749: saving model to model2_3_1_chkp\weights.749-1.95.hdf5
Epoch 750/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00750: saving model to model2_3_1_chkp\weights.750-1.95

303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00786: saving model to model2_3_1_chkp\weights.786-1.95.hdf5
Epoch 787/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00787: saving model to model2_3_1_chkp\weights.787-1.95.hdf5
Epoch 788/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1432 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00788: saving model to model2_3_1_chkp\weights.788-1.95.hdf5
Epoch 789/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00789: saving model to model2_3_1_chkp\weights.789-1.95.hdf5
Epoch 790/800
303/303 [==============================] - 7s 23ms/step - loss: 1.9459 - acc: 0.1436 - val_loss: 1.9459 - val_acc: 0.1436

Epoch 00790: saving model to model2_3_1_chkp\weights.790-1.95